In [20]:
import math
from pymavlink import mavutil
import time

In [21]:


robot = 0

def initConnection():
    protocol = 'tcp'; address = '192.168.2.2'; port = '6777'; connection = f'{protocol}:{address}:{port}'


    #Create connection from topside
    robot = mavutil.mavlink_connection(connection)

    #verify connection
    robot.wait_heartbeat()
    return robot

def armRobot():
    print("Arming robot")

    robot.arducopter_arm()
    robot.motors_armed_wait()
    print("Robot armed")
    
def setMode(mode):
    if mode not in robot.mode_mapping():
        print('Error, unknown mode')
        disarmRobot()
    else:
        mode_id = robot.mode_mapping()[mode]
        robot.set_mode(mode_id)

def disarmRobot():
    print("Disarming Robot")
    robot.arducopter_disarm()


In [22]:
#each RC Channel corresponds to a different DOF

channel = {
    "pitch": 1,
    "roll" : 2,
    "vertical": 3,
    "yaw": 4,
    "forward": 5,
    "lateral": 6, 
    "pan": 7,
    "tilt": 8,
    "lights1": 9,
    "lights2": 10
}



In [23]:

def set_rc_channel_pwm(channel_id, pwm=1500):
    """ Set RC channel pwm value
    Args:
        channel_id (TYPE): Channel ID
        pwm (int, optional): Channel pwm value 1100-1900
    """
    if channel_id < 1 or channel_id > 18:
        print("Channel does not exist.")
        return

    # Mavlink 2 supports up to 18 channels:
    # https://mavlink.io/en/messages/common.html#RC_CHANNELS_OVERRIDE
    rc_channel_values = [65535 for _ in range(18)]
    rc_channel_values[channel_id - 1] = pwm
    robot.mav.rc_channels_override_send(
        robot.target_system,                # target_system
        robot.target_component,             # target_component
        *rc_channel_values)  


def stopMotion():
    for i in range(1, 6):
        set_rc_channel_pwm(i, 1500)

def goForward(pwm):
    id = channel.get("forward")
    set_rc_channel_pwm(id, pwm)

def goVertical(pwm):
    id = channel.get("vertical")
    set_rc_channel_pwm(id, pwm)

def turn(pwm):
    id = channel.get("turn")
    set_rc_channel_pwm(id, pwm)

def roll(pwm):
    id = channel.get("roll")
    set_rc_channel_pwm(id, pwm)

def strafe(pwm):
    id = channel.get("lateral")
    set_rc_channel_pwm(id, pwm)

def cameraControl(angle):
    if(angle > 70 or angle < -30):
        print("target camera angle out of bounds")
        return
    currTilt = getCameraTilt()
    currAngle = tilt2Angle(currTilt)
    tol = 1
    id = channel.get("tilt")
    error = angle - currAngle
    absError = abs(error)

    currTilt = getCameraTilt()
    currAngle = tilt2Angle(currTilt)
    error = angle - currAngle
    absError = abs(error)
    if(absError) <= tol:
        set_rc_channel_pwm(id, 1500)
        return
                
def setGain(gain=0.3):
    param = b'PILOT_GAIN'
    robot.mav.param_set_send(
        robot.target_system,
        robot.target_component,
        param,
        gain,
        mavutil.mavlink.MAV_PARAM_TYPE_REAL32
    )

import threading
def cameraControlAsync(target_deg, kp=8.0, tol_deg=1.0, period=0.05):
    def update():
        id = channel.get("tilt")
        curr_deg = tilt2Angle(getCameraTilt())
        err = target_deg - curr_deg

        if abs(err) <= tol_deg:
            set_rc_channel_pwm(id, 1500)
            return

        pwm = int(1500 + kp * err)
        pwm = max(1300, min(1700, pwm))
        set_rc_channel_pwm(id, pwm)

        threading.Timer(period, update).start()

    update()


def tilt2Angle(tilt):
    print(tilt)
    tilt = tilt - 0.3
    return tilt * 100 #deg

def getCameraTilt():
    robot.mav.command_long_send(
    robot.target_system,
    robot.target_component,
    mavutil.mavlink.MAV_CMD_REQUEST_MESSAGE,
    0,
    251, 0, 0, 0, 0, 0, 0
)
    msg = robot.recv_match(type="NAMED_VALUE_FLOAT")
    if msg:
        name = msg.name.decode('utf-8') if isinstance(msg.name, bytes) else msg.name
        value = msg.value
        if name == 'CamTilt':
            print(value)
            return value
        else: 
            return getCameraTilt()
    else:
        return getCameraTilt()



In [24]:
robot = initConnection()


In [25]:
def lights(pwm):
    id = channel.get("lights1")
    set_rc_channel_pwm(id, pwm)

def lightsOn():
    lights(1900)

def lightsOff():
    lights(1100)



In [56]:
lightsOff()

In [41]:
# robot = initConnection()
mode = "MANUAL"
# armRobot()
setMode(mode)
setGain(0.3)
# cameraControl(45)
while False:
    goVertical(1700)
# time.sleep(5)
# goVertical(1700) #> 1500 is up
# time.sleep(2)
# stopMotion()






KeyboardInterrupt: 

In [43]:
stopMotion()

In [57]:
disarmRobot()

Disarming Robot
